# Dewan Lab Image Analysis

In [ ]:
import os
os.environ['ISX'] = '0'

from tqdm.notebook import tqdm
import numpy as np
import pandas as pd
from dewan_calcium.helpers import IO, parse_json
from dewan_calcium.helpers.project_folder import ProjectFolder
from dewan_calcium import classifiers
pd.options.mode.copy_on_write = "warn"

print('Finished importing required libraries!')

In [ ]:
# Create Project Folder to Gather and Hold all the File Paths
folder_dir = r'D:\Test_Data\Combined\Identity'
project_folder = ProjectFolder('ODOR', project_dir=folder_dir, combined=True)

In [ ]:
# If this is the first time the project folder has been created,
# move the files to the appropriate directories and then run this cell, otherwise skip this cell

project_folder.get_data()

In [ ]:
combined_data = pd.read_pickle(project_folder.raw_data_dir.combined_data_path, compression={'method': 'xz'})
cell_names = combined_data.columns.get_level_values(0).values
trial_labels = np.unique(combined_data.columns.get_level_values(1).values)

In [ ]:
mean_score_df, split_score_df, all_confusion_mats = classifiers.single_neuron_decoding(combined_data)

In [ ]:
from sklearn.metrics import ConfusionMatrixDisplay

cm = all_confusion_mats[2]

ConfusionMatrixDisplay(cm).plot()